# Routage

## Sommaire

1. [Introduction](#introduction)
1. [Bases du routage dans Laravel](#bases)
    1. [Fichiers de routage](#files)
    1. [Méthodes HTTP et routes](#methodes)
        1. [Requêtes HTTP](#requetes)
        1. [Redirections HTTP](#redirections)
        1. [Vues](#vues)
1. [Options de routes](#options)
    1. [Nommer les routes](#nommage)
    1. [Paramètres de routes](#parametres)
        1. [Contraintes globales](#contraintes)
    1. [Préfixes de routes](#prefixes)
    1. [Gestion des erreurs](#erreurs)

<a id='introduction'></a>
## Introduction

Le routage est une phase essentielle des applications web. Celles-ci sont en effet définies par leur **API**, qui est constituée par l'ensemble des URI accessibles via des requêtes HTTP.

> <div class="alert alert-block alert-info">
> <b>Note :</b> Il existe par ailleurs des outils pour spécifier l'API d'une application web, comme <b>Swagger</b>, mais ce n'est pas vraiment l'objet du cours.
> </div>

Rappelons que l'application est (généralement) accessible par un unique point d'entrée, appelé **contrôleur principal**. Ce point d'entrée est (là aussi, généralement, par convention) le script `public/index.php`. D'une manière générale, c'est le serveur HTTP, comme Apache ou Nginx, qui se charge de renvoyer la requête vers le point d'entrée par le biais d'un mécanisme de **réécriture d'URL**.

Rappelons aussi que, par mesure de sécurité, le dossier `public` doit être le seul accessible depuis le monde extérieur.

Une fois la phase d'initialisation de l'application terminée, la plate-forme devra interpréter la requête qui lui est transmise. Contrairement à une page web statique dont la (représentation de la) ressource est directement liée à un URL, les ressources délivrées par Laravel sont « virtuelles », au sens où elles calculées à chaque requête de l'utilisateur. La phase de routage consiste donc décoder la requête pour trouver quelle fonction doit effectuer ce calcul. Elle est donc un moment spécifique qui vient s'intercaler entre l'événement utilisateur et la prise en charge de la requête par un **contrôleur**.

<a id='bases'></a>
## Bases du routage dans Laravel

Avec Laravel, l'ensemble du routage est implémenté de manière _fonctionnelle_, c'est-à-dire que les routes sont des fonctions associées à des **fonctions de rappel** (ou « callbacks » en anglais). C'est à peu près la même forme que ce que l'on trouve en JavaScript avec `fetch` (ou pour les plus anciens, les requêtes AJAX à la `jQuery`).

Une route est donc définie principalement trois éléments :
- une **méthode HTTP**, qui indique quel type de requête que l'on s'attend à traiter
- un **schéma d'URL** qui précise la ressource qui est attendue par l'utilisateur
- une **fonction de rappel** qui spécifie comment la requête doit être traitée et, éventuellement, ce qui doit être renvoyé au client.

Sous sa forme la plus simple, elle se déclare ainsi :

In [1]:
%%php

use Illuminate\Support\Facades\Route;
 
Route::get('/greeting', fn () => 'Hello World');

UsageError: Cell magic `%%php` not found.


Nous retrouvons :
- la méthode `GET`, qui correspond à la méthode statique `get` de la classe `Route` ;
- un URL (`/greeting` pour `http://<domaine>/greeting`) qui est transmis par le client
- une simple fonction anonyme qui rend une chaîne de caractères constituant la **réponse** (au sens HTTP du serveur à la requête.

### Fichiers de routage

L'ensemble de l'API de l'application est définie dans des fichiers de routage qui sont de simples scripts PHP. Ceux-ci résident dans le dossier `routes`, à la racine de l'application.

Dans ce dossier, on trouve quatre fichiers différents, qui correspondent à quatre types de requêtes :
1. `web` : le groupe de routes le plus courant, représentant des requêtes émises par des clients web, comme des navigateurs ; le groupe `web` autorise la gestion de sessions et de protection CSRF
2. `api` : groupe de routes représentant des requêtes « sans état » (_stateless_, c'est-à-dire strictement REST)
3. `channels` : groupe de route pour (principalement) la diffusion de notification en mode « _broadcast_ » vers un ensemble d'écouteurs
4. `console` : groupe de routes destiné à exécuter des commandes depuis la ligne de commande

Pour le moment, nous nous intéresserons principalement aux routes du groupe `web`.

Comme nous pouvons le voir dans le fichier d'exemple, nous n'avons rien d'autre à faire que déclarer les routes.

### Méthodes de routes

Les routes peuvent être réparties en plusieurs catégories.

#### Requêtes HTTP

En premier lieu, celles qui correspondent à des requêtes HTTP. De même que nous avons vu l'emploi de `GET`, il existe des outils pour appréhender les autres méthodes HTTP 

In [ ]:
%%php

Route::get($uri, $callback);
Route::post($uri, $callback);
Route::put($uri, $callback);
Route::patch($uri, $callback);
Route::delete($uri, $callback);
Route::options($uri, $callback);

Selon les serveurs HTTP, il peut être plus ou moins facile d'indiquer quelle méthode HTTP est employée. De plus les navigateurs web ne savent globalement gérer que des `GET` et des `POST`. Nous verrons par la suite, lors de la question des formulaires, comment indiquer à Laravel que nous voulons une méthode précise, conforme aux recommandations de REST.

Quelquefois, nous aimerions qu'une requête soit associée à plusieurs méthodes HTTP alternatives. Dans ce cas, on peut utiliser `match`, voire `any` qui autorise, comme son nom l'indique, n'importe quelle méthode.

%%php

Route::match(['get', 'post'], '/book', fn() => 'Hello world');
Route::any('/book', fn() => 'Hello world');

> <div class="alert alert-block alert-info">
> <b>Note :</b> L'ordre de déclaration des routes n'est pas anondin, car Laravel prendra la première route dont le schéma d'URL correspond à la requête entrante.
> </div>

#### Redirections HTTP

Une catégorie spécifique traite des redirections, c'est-à-dire des réponses de statut 30x. Pour cela, on utilise la fonction `redirect` :

In [ ]:
%%php

Route::redirect('/url_1', '/url_2', <code de statut>);

// permanentRedirect renvie implicitement le code 301
Route::permanentRedirect('/url_1', '/url_2');

#### Vues

La troisième catégorie de routes est celle qui faire référence à des **vues**.

> <div class="alert alert-block alert-info">
> <b>Note :</b> Pour plus de détails sur les vues et le moteur de rendu <b>Blade</b>, reportez-vous au document afférent.
> </div>

On peut utiliser la fonction de support `view` pour faire référence à un fichier statique. Par exemple :

In [ ]:
%%php

Route::view('/about', 'about', ['day' => (new Datetime())->format('d/m/Y')]);

Dans cet exemple :
- le premier argument est un schéma d'URL
- le second argument est le nom du fichier contenant la page à afficher
- le troisième argument permet de passer des variables qui seront insérées dans la page par le moteur de rendu.

## Options de routes

Nous savons maintenant comment créer des routes simples et afficher du contenu dans l'écran du navigateur. Mais, naturellement, nous voyons bien que beaucoup de choses manquent ou sont difficilement exploitables. Comment améliorer nore système de routage ?

### Nommer les routes

Donner une « étiquette » à une route est très important. En utilisant les étiquettes dans le code au lieu de faire directement référenc à la route, nous allons nous abstraire du code et augmenter la résilience de l'application en cas de modification de la politique d'URL, par rexemple. Nommer une route est très simple :

In [ ]:
Route::get('/greeting', fn () => 'Hello World';)->name('greeting');

Grâce au nom que nous donnons à la route, nous pourrons désormais engendrer des URL dans le code de l'application, par exemple pour insérer des liens hypertextes dans les pages HTML.

In [ ]:
%%php

$url = route('greeting');

Désormais, si je souhaite changer l'URL de `/greeting` en `/bienvenue`, cela reste transparent du pont de vue du code de l'application.

De même nous pouvons améliorer nos redirections HTTP :

In [ ]:
%%php

return redirect()->route('greeting');

> <div class="alert alert-block alert-info">
> <b>Note :</b> En termes de bonnes pratiques, toutes les routes devraient être nommées. Naturellement, deux routes ne pourront pas porter le même nom, au risque de créer des conflits de nommage et des erreurs dans le traitement des requêtes.
> </div>

### Paramètres de routes

Dans de nombreux cas, évidemment, nous aurons besoin de capturer la notion de « _famille de routes_ ». Dans une application de commerce en ligne, par exemple, on a besoin d'une page pour afficher chaque produit, identifié par sa référence. Nous devons donc avoir un moyen d'introduire des variables dans les schémas d'URL.

Rien de plus simple :

In [ ]:
%%php

Route::get('/product/{id}', function ($id) {
    return 'Produit n° '.$id;
});

Comme on le voit :
- Les parties variables d'URL sont entourées d'accolades (les noms de variables peuvent contenir des `_`) ;
- La variable `id` est réutilisée comme paramètre de la fonction de rappel.

On peut apporter quelques précisions sur le fonctionnement de ces paramètres :
- Il est possible de définir autant de paramètres que nécessaire dans le schéma d'URL
- La correspondance entre le schéma d'URL et la liste des paramètres de la fonction de rappel se fait en fonction de leur **position** (elles peuvent donc ne pas porter le même nom (même si cela ne semble pas une bonne pratique)
- Il est possible de rendre un paramètre optionnel en ajoutant un `?` après le nom du paramètre dans l'URL... et en définissant une valeur par défaut dans la fonction de rappel
- Il est possible de définir des contraintes sur les paramètres, en particulier basées sur des expressions régulières.

Voici un petit exemple synthétisant ces possibilités :

In [ ]:
%%php

Route::get('/post/{pid}/comment/{cid?}', function ($postId, $commentId = 'all') {
    return 'Commentaires de l’article n° '.$postId;
})->where('pid', '[0-9]+')
  ->where('cid', '[0-9]+|all');

Dans notre cas, nous définissons que si l'identitiant d'un commentaire n'est pas précisé, l'application doit les afficher tous.


> <div class="alert alert-block alert-info">
> <b>Note :</b> Nous verrons un peu plus loin au moment de l'étude des contrôleurs et des modèles deux autres mécanismes des routes :
> - l'**injection de dépendances, qui permet de d'importer des classes dans les contrôleurs (entre autres)
> - la **conversion de paramètres**, qui permet d'importer un objet directement à partir de son identifiant
> </div>

#### Contraintes globales

Dans le cas où un paramètre d'une route aurait toujours la même forme, il est possible de définir cette contrainte globalement, et non pas pour chaque route.

Pour cela, nous devons faire appel à un type de classes particulier, appelé `Provider` ou **fournisseur de service**. Ces classes se trouvent dans le dossier `/app/Providers`.

> <div class="alert alert-block alert-info">
> <b>Note :</b> Les fournisseurs de services sont le point central de l'amorçage de toutes les applications Laravel. C'est-à-dire la pahse où il s'agit d'enregistrer des objets requis par l'application, comme des liaisons de conteneurs de services, des écouteurs d'événements, des intergiciels et même des routes. Si vous ouvrez le fichier `config/app.php`, vous verrez un tableau de ces fournisseurs. Il s'agit de toutes les classes qui seront chargées pour votre application. Par défaut, un ensemble de fournisseurs de services de base de Laravel est répertorié dans ce tableau. Ces fournisseurs amorcent les composants de base de Laravel, tels que le mailer, la file d'attente, le cache, etc. Beaucoup de ces fournisseurs sont des fournisseurs "différés", ce qui signifie qu'ils ne seront pas chargés à chaque demande, mais seulement lorsque les services qu'ils fournissent sont réellement nécessaires.
> </div>

Parmi toutes les classes, on trouve `RouteServiceProvider`, qui est l'endroit où nouspouvons définir des contraintes globales pour les paramètres de routes :

In [ ]:
public function boot()
{
    Route::pattern('id', '[0-9]+');
}

### Préfixes de routes

Les routes peuvent être groupées, ce qui permet de faciliter certains aspects de leur définition. Comme nous allons le voir par la suite, les groupes sont liés à la notion de « _middleware_ » (ou intergiciel) et de contrôleur. Pour ce qui concerne les routes elles-mêmes, il est possible de modifier la structure de tout un ensemble d'URL.

In [ ]:
%%php

Route::prefix('admin')->group(function () {
    Route::get('/users', function () {
        // Matches The "/admin/users" URL
    });
});
        
Route::name('admin.')->group(function () {
    Route::get('/users', function () {
        // Route assigned name "admin.users"
    })->name('users');
}); 

- Dans le premier cas, c'est le schéma des URL qui est modifié
- Dans le second cas, c'est le nom des URL qui est modifié

On trouve un exemple très parlant dans la configuration par défaut de Laravel, qui regroupe des reoutes sur la base du nom du fichier dans lequel elles sont écrites. Dans le fournisseur de services `RouteServiceProvder` :

In [ ]:
%%php

Route::prefix('api')
    // ...
    ->group(base_path('routes/api.php'));

La fonction `base_path` récupère le le chemin de système de fichiers d'un certain fichier PHP, ici le fichier de routage pour l'accès à l'application par API. Toutes les routes définies dans le fichier de routage sont ensuite préfixées par le segment `api`.

### Gestion des erreurs

En utilisant la méthode `Route::fallback`, vous pouvez définir une route qui sera exécutée lorsqu'aucune autre route ne correspond à la requête entrante. En règle générale, les requêtes non traitées renvoient automatiquement une page "404" via le gestionnaire d'exceptions de votre application. Cependant, comme vous définissez généralement la route de repli dans votre fichier routes/web.php, tous les intergiciels du groupe d'intergiciels Web s'appliqueront à cette route. Vous êtes libre d'ajouter des intergiciels supplémentaires à cette route si nécessaire :

In [3]:
%%php

Route::fallback(function () {
    return 'Aucune ressource ne correspond à votre demande'
});

UsageError: Cell magic `%%php` not found.


Cette méthode devrait naturellement être la dernière definie par votre application.